# ECT Report Analysis
## Description of Report
Quoting the description on the report home page:
>"Average time in days it takes for a Customers issue to be resolved measured from the day the Customer contacts the Call Center and a Service Order is opened unitl the case status is moved to closed. Calculation includes FOD, CRU, On-Site and Depot Service Orders."

The SLA goal is 5.60 days.

## Method for KPI Calculation
The report performs a simple calculation to determine whether or not a case met the SLA. If the case is open for less than 5.60 days then it is marked as a `Make`; otherwise it will be a `Miss`. This calculation is performed by subtracting the values in columns `SR Close TS` and `SR Create TS`.

The KPI for this metric is calculated by the mean value of the subtraction of `SR Close TS` and `SR Create TS` for each row in the data set.

## Limitations of the Report
* By default this report measures *all* available data regardless of case age. Therefore the KPI reflects *all* cases since the reporting tool started capturing data. This is the number shown in the dashboard.
* The column `SR Close TS` has the value of the case's final closure date. This value is not reflective of the performance of the call center as this event is triggered when all work on the case is completed (CX status). A few examples for possible delays outside of the call center's control:
    * Limited part availability.
    * Customer delaying service (additional information needed, not available).
    * Technician not available to service machine.
    * Machine lost in transit.
    * Carrier unable to deliver a part.
* Changing the date range on the report filters the data by the values in the `SR Close TS` column. For example, we may pick all cases YTD for our values, and the report will pull cases which were opened prior to the current year if the `SR Close TS` value is on the current year.
* There is no true indicator of call center performance in the data columns. The best indicator would be the time when the first action plan (AP) is placed on a case. This metric is not available as of writing this report.
## Conclusion
The ECT Report does not serve as an objective measure of the performance of the call center due to its inherent limitations. The case closure time is the determinant factor into whether or not the KPIs are met. This number is not reflective of the actions performed by the call center. A better measure would be the first AP time on the case. Delays in the field can cause cases to remain open and for the closure time to be delayed indefinitely.

In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [68]:
#Read CSV file
df = pd.read_csv('Resources/ECT_Data.csv', low_memory=False)
df[['SR Create TS', 'SR Close TS', 'Cust Contact TS']] = df[['SR Create TS', 'SR Close TS', 'Cust Contact TS']].astype('datetime64[ns]')

In [69]:
#Create data frames split by years
combined_df = df
ytd_df = df[df['SR Create TS'] >= '2020-01-01']
prevy_df = df[df['SR Create TS'] < '2020-01-01']

In [70]:
#Create new data frames with columns we're interested in
combined_datetime_df = combined_df[['OCPM #', 'Make/Miss', 'SR Create TS', 'SR Close TS', 'Cust Contact TS']]
ytd_datetime_df = ytd_df[['OCPM #', 'Make/Miss', 'SR Create TS', 'SR Close TS', 'Cust Contact TS']]
prevy_datetime_df = prevy_df[['OCPM #', 'Make/Miss', 'SR Create TS', 'SR Close TS', 'Cust Contact TS']]

#Create new columns with difference on data frames
combined_datetime_df['Close and Create Diff'] = (combined_datetime_df['SR Close TS'] - combined_datetime_df['SR Create TS'])
ytd_datetime_df['Close and Create Diff'] = (ytd_datetime_df['SR Close TS'] - ytd_datetime_df['SR Create TS'])
prevy_datetime_df['Close and Create Diff'] = (prevy_datetime_df['SR Close TS'] - prevy_datetime_df['SR Create TS'])


C:\Users\jechevarria\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\jechevarria\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jechevarria\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [71]:
#Create groups
combined_make_miss_group = combined_datetime_df.groupby('Make/Miss')
ytd_make_miss_group = ytd_datetime_df.groupby('Make/Miss')
prevy_make_miss_group = prevy_datetime_df.groupby('Make/Miss')

#New data frame for summary of Make/Miss percentages
percent_make_miss_df = pd.DataFrame({    
    '2020': ytd_make_miss_group['OCPM #'].count() * 100 / ytd_make_miss_group['OCPM #'].count().sum(),
    '2019': prevy_make_miss_group['OCPM #'].count() * 100 / prevy_make_miss_group['OCPM #'].count().sum(),
    'Combined': combined_make_miss_group['OCPM #'].count() * 100 / combined_make_miss_group['OCPM #'].count().sum(),
})

#Format columns with percentages
percent_make_miss_df['2020'] = percent_make_miss_df['2020'].map('{:.2f}%'.format)
percent_make_miss_df['2019'] = percent_make_miss_df['2019'].map('{:.2f}%'.format)
percent_make_miss_df['Combined'] = percent_make_miss_df['Combined'].map('{:.2f}%'.format)

#Display data frame
percent_make_miss_df

,2020,2019,Combined
Make/Miss,,,
Make,86.14%,24.64%,36.67%
Miss,13.86%,75.36%,63.33%


In [72]:
#New data frame for counts
count_make_miss_df = pd.DataFrame({
    '2020 Count': ytd_make_miss_group['Make/Miss'].count(),    
    '2019 Count': prevy_make_miss_group['Make/Miss'].count(),
    'Combined Count': combined_make_miss_group['Make/Miss'].count()
})

#Display data frame
count_make_miss_df

,2020 Count,2019 Count,Combined Count
Make/Miss,,,
Make,3971,4670,8641
Miss,639,14283,14922


In [73]:
#Descriptive stats data frame

describe_df = pd.DataFrame({
    '2020': ytd_datetime_df['Close and Create Diff'].describe(),
    '2019': prevy_datetime_df['Close and Create Diff'].describe(),
    'Combined': combined_datetime_df['Close and Create Diff'].describe()
})

describe_df

,2020,2019,Combined
count,4610,18953,23563
mean,3 days 09:20:36.442516,14 days 11:20:52.459241,12 days 07:18:18.988244
std,2 days 07:37:34.000030,10 days 21:48:51.613503,10 days 18:36:15.472835
min,0 days 00:01:00,0 days 00:02:00,0 days 00:01:00
25%,1 days 07:14:15,5 days 21:13:00,4 days 02:34:00
50%,3 days 02:03:30,13 days 04:34:00,8 days 10:18:00
75%,4 days 09:49:00,20 days 18:09:00,18 days 02:44:00
max,12 days 07:19:00,166 days 23:18:00,166 days 23:18:00
